<a href="https://colab.research.google.com/github/xgabiez/Ciencia_de_Dados_com_Python_IMERSAO_SANTANDER/blob/main/ProcessoPipelineETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [2]:
import pandas as pd

df = pd.read_csv('SDW2025.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[101, 102, 103]


In [3]:
import json

with open('users_mock.json', 'r') as f:
    all_users = json.load(f)

users = [user for user in all_users if user["id"] in user_ids]
print(json.dumps(users, indent=2, ensure_ascii=False))


[
  {
    "id": 101,
    "name": "Gabrielly",
    "account": {
      "id": 201,
      "number": "12345-1",
      "agency": "0001",
      "balance": 5000.0,
      "limit": 1500.0
    },
    "card": {
      "id": 301,
      "number": "**** **** **** 9876",
      "limit": 5000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 102,
    "name": "Marcos",
    "account": {
      "id": 202,
      "number": "54321-2",
      "agency": "0001",
      "balance": 1200.0,
      "limit": 500.0
    },
    "card": {
      "id": 302,
      "number": "**** **** **** 1234",
      "limit": 1500.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 103,
    "name": "Ana Paula",
    "account": {
      "id": 203,
      "number": "67890-3",
      "agency": "0001",
      "balance": 700.0,
      "limit": 300.0
    },
    "card": {
      "id": 303,
      "number": "**** **** **** 5678",
      "limit": 800.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [12]:
import json

# 1. Carregar o arquivo JSON original
with open("users_mock.json", "r", encoding="utf-8") as f:
    users = json.load(f)

# 2. Mensagens personalizadas
messages = {
    "Gabrielly": "Gabrielly, seu saldo está em um ótimo nível! Que tal começar a investir uma parte para fortalecer ainda mais seu futuro financeiro?",
    "Marcos": "Marcos, você está mantendo um bom controle financeiro! Avalie investir uma parte do seu saldo para começar a construir uma reserva sólida.",
    "Ana Paula": "Ana Paula, pequenos passos fazem grande diferença. Que tal iniciar um investimento leve para fazer seu dinheiro começar a trabalhar por você?"
}

# 3. Criar o campo news para cada usuário
for index, user in enumerate(users):
    mensagem = messages.get(
        user["name"],
        "Aproveite nossas novas opções de investimento!"
    )

    user["news"] = [
        {
            "id": index + 1,
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/investments.svg",
            "description": mensagem
        }
    ]

# 4. Salvar o arquivo transformado
with open("user_mock_transformed.json", "w", encoding="utf-8") as f:
    json.dump(users, f, indent=2, ensure_ascii=False)

# 5. Exibir o JSON completo transformado — estrutura final
print("=== JSON TRANSFORMADO ===\n")
print(json.dumps(users, indent=2, ensure_ascii=False))



=== JSON TRANSFORMADO ===

[
  {
    "id": 101,
    "name": "Gabrielly",
    "account": {
      "id": 201,
      "number": "12345-1",
      "agency": "0001",
      "balance": 5000.0,
      "limit": 1500.0
    },
    "card": {
      "id": 301,
      "number": "**** **** **** 9876",
      "limit": 5000.0
    },
    "features": [],
    "news": [
      {
        "id": 1,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/investments.svg",
        "description": "Gabrielly, seu saldo está em um ótimo nível! Que tal começar a investir uma parte para fortalecer ainda mais seu futuro financeiro?"
      }
    ]
  },
  {
    "id": 102,
    "name": "Marcos",
    "account": {
      "id": 202,
      "number": "54321-2",
      "agency": "0001",
      "balance": 1200.0,
      "limit": 500.0
    },
    "card": {
      "id": 302,
      "number": "**** **** **** 1234",
      "limit": 1500.0
    },
    "features": [],
    "news": [
      {
        "id": 2,
        "

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [17]:
import json

# Carregar usuários transformados
with open("user_mock_transformed.json", "r", encoding="utf-8") as f:
    users = json.load(f)

# Função para simular um PUT na API
def update_user(user):
    # Aqui apenas exibimos o usuário atualizado
    # como se o PUT tivesse sido feito
    print(f"{user['id']}")
    return True  # sempre retorna sucesso

# Executar "envio" do load
print("=== LOAD: atualizando usuários no sistema final ===\n")
for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!\n")

# Salvar como arquivo final da pipeline
with open("user_mock_final.json", "w", encoding="utf-8") as f:
    json.dump(users, f, indent=2, ensure_ascii=False)

print("Arquivo 'user_mock_final.json' salvo com sucesso!")


=== LOAD: atualizando usuários no sistema final ===

101
User Gabrielly updated? True!

102
User Marcos updated? True!

103
User Ana Paula updated? True!

Arquivo 'user_mock_final.json' salvo com sucesso!
